Our object detection system, called Faster R-CNN, is composed of two modules. The first module is a deep
fully convolutional network that proposes regions, and the second module is the Fast R-CNN detector ... that uses the proposed regions. The entire system is a single, unified network for object detection (Figure 2).

<img src="frcnn.png" width=50% height=50%/>

# Region Proposal Networks

A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score. We model this process with a fully convolutional network, which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network, we assume that both nets share a common set of convolutional layers. 

> *Both the RPN and the Fast R-CNN detection network share a "backbone" feature extractors whose feature maps are used for the subsequent steps. In the original Faster R-CNN paper the extract feature maps from just one layer [TODO: confirm] but later works use feature pyramids. We subsequently discuss how feature pyramids are extracted.*

To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an n × n spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature. This feature is fed into two sibling fully- connected layers—a box-regression layer (reg) and a box-classification layer (cls). We use n = 3 in this paper, noting that the effective receptive field on the input image is large. This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an $n \times n$ convolutional layer followed by two sibling $1 \times 1$ convolutional layers (for reg and cls, respectively).

At each sliding-window location, we simultaneously predict multiple region proposals, where the number of maximum possible proposals for each location is denoted as $k$. So the reg layer has 4k outputs encoding the coordinates of $k$ boxes, and the cls layer outputs $2k$ scores that estimate probability of object or not object for each proposal4. The $k$ proposals are parameterized relative to $k$ reference boxes, which we call ***anchors***. An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left) [TODO: add this] ... For a convolutional feature map of a size $W \times H$ ... there are $WHk$ anchors in total.

In [2]:
def rpn_head(inputs, num_anchors, filters=256, kernel_size=3):
    # (B, H, W, 256)
    inputs = tf.layers.conv2d(inputs, filters=filters, kernel_size=kernel_size, padding="same")
    # (B, H, W, 4*A)
    deltas = tf.layers.dense(inputs, units=4 * num_anchors)
    # (B, H, W, A)
    scores = tf.layers.dense(inputs, units=num_anchors)
    # (B, H*W*A, 4)
    deltas = tf.reshape(deltas, [tf.shape(deltas)[0], -1, 4])
    # (B, H*W*A)
    scores = tf.reshape(deltas, [tf.shape(deltas)[0], -1])
    return deltas, scores



An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left). By default we use 3 scales and 3 aspect ratios, yielding k = 9 anchors at each sliding position.

> Each pixel in the input feature map corresponds to an anchor box. 

In [4]:
#TODO: Add code here to generate anchor boxes 

In [5]:
#TODO: Add their observations about single scale and what is said in FPNs

# Digression on Feature Pyramid Networks

Our goal is to leverage a ConvNet’s pyramidal feature
hierarchy, which has semantics from low to high levels, and build a feature pyramid with high-level semantics through- out. The resulting Feature Pyramid Network is general- purpose and in this paper we focus on sliding window proposers (Region Proposal Network, RPN for short) [29] and region-based detectors (Fast R-CNN) [11]. We also generalize FPNs to instance segmentation proposals in Sec. 6. Our method takes a single-scale image of an arbitrary size as input, and outputs proportionally sized feature maps at multiple levels, in a fully convolutional fashion. This pro- cess is independent of the backbone convolutional architectures (e.g., [19, 36, 16]), and in this paper we present results using ResNets [16]. The construction of our pyramid involves a bottom-up pathway, a top-down pathway, and lat- eral connections, as introduced in the following.